<a href="https://colab.research.google.com/github/Jcc329/Jessica_DATA606/blob/main/Notebooks/5.Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data 606 Capstone
## Notebook 5
## Phase 2 Part 1: Topic Modeling
Jessica Conroy

<b> Goal: </b> 
This Notebook aims to perform topic modeling to identify key topics and differences in those topics among the pre covid and during covid time frames. 
1. LDA and NMF (non-negative matrix factorization) from game description, reviews, tags
2. Text similarity analysis

In [1]:
!pip install gensim==3.8.0

In [72]:
pip install googletrans==3.1.0a0

  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=a8da55423c1f21e6773f52664a80dcf75a8505d2d06523d74706375d095f3eb6
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [1]:
#Import libraries
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import textwrap

import gensim
print(gensim.__version__)
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LsiModel
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import preprocess_documents
from io import TextIOWrapper

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
# print(stopwords.words('english'))


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

from googletrans import Translator

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

3.8.0
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#Load Data
# GameData = pd.read_csv('./FinalSteamData.csv')
GameData = pd.read_csv('./FinalGameData_postProcessing.csv')
# GameData.head()

# Perform some pre-requisite text processing

In [5]:
#Remove digits from tags. 

def removedigit(s):
    try:
        s = re.sub("\d+", "", s)
    except: 
        s
    return s
GameData['tags.1'] = GameData['tags.1'].apply(removedigit)
GameData['tags.1']

0        action  casual  arcade  shoot em up  d  colorf...
1                                                      NaN
2                                                      NaN
3        bullet hell  coop  shoot em up  replay value  ...
4        action  shooter  action rpg  shoot em up  rpg ...
                               ...                        
14947    adventure  indie  point  click  puzzle  short ...
14948                                                  NaN
14949                                   adventure  casual 
14950                                                  NaN
14951                                                  NaN
Name: tags.1, Length: 14952, dtype: object

In [6]:
#Lemmatize the text (Lemmatisation in linguistics is the process of grouping together the inflected forms of a word so they can be analysed as a single item)
# Topic modeling functions are informed and guided by another project I worked on https://github.com/Jcc329/Text_Summarization/blob/main/notebooks/Topic_Modeling_YunpengLi.ipynb
# used methods for lemmatizing found here https://www.geeksforgeeks.org/python-lemmatization-approaches-with-examples/

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None
def nltk_lemmatize(s):
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(str(s))) 
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:       
            # else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    lemmatized_sentence = " ".join(lemmatized_sentence)
    return lemmatized_sentence
GameData['tags.1'] = GameData['tags.1'].apply(nltk_lemmatize)
# GameData['tags.1']
GameData['detailed_description.1'] = GameData['detailed_description.1'].apply(nltk_lemmatize)
# GameData['detailed_description.1']
GameData['Top Reviews by Upvotes.1'] = GameData['Top Reviews by Upvotes.1'].apply(nltk_lemmatize)
# GameData['Top Reviews by Upvotes.1']

In [7]:
#remove additional stopwords
stop_words = set(stopwords.words("english"))
stop_words = stop_words.update(["game",'games', "player", 'play'])

def removestopwords(s):
    #Remove stop words and sent to lowercase
    try:
        s = " ".join([word.lower() for word in str(s).split() if word.lower() not in stop_words])
    except:
        s
    return s

GameData['tags.1'] = GameData['tags.1'].apply(removestopwords)
# GameData['tags.1']
GameData['detailed_description.1'] = GameData['detailed_description.1'].apply(removestopwords)
# GameData['detailed_description.1']
GameData['Top Reviews by Upvotes.1'] = GameData['Top Reviews by Upvotes.1'].apply(removestopwords)
# GameData['Top Reviews by Upvotes.1']

In [9]:
# Filter out non english text
#https://pypi.org/project/googletrans/
#https://stackoverflow.com/questions/57282711/google-translate-library-is-printing-googletrans-models-translated-at-0x1eaf9bf

def taglanguage(df,col):
    lang_tag = []
    for s in df[col]:
        translator = Translator()
        result = translator.detect(s[0:100])
        lang_tag.append(result.lang)
    # print(lang.detect_language())
    return lang_tag

GameData['tags.Lang'] = taglanguage(GameData, 'tags.1')
# GameData['tags.1']
GameData['detailed_description.Lang'] =  taglanguage(GameData, 'detailed_description.1')
# GameData['detailed_description.1']
GameData['Top Reviews by Upvotes.Lang'] =  taglanguage(GameData, 'Top Reviews by Upvotes.1')
# GameData['detailed_description.Lang']
GameData.to_csv('./FinalGameData_postProcessing.csv')


0        en
1        en
2        en
3        en
4        en
         ..
14947    en
14948    en
14949    en
14950    en
14951    en
Name: detailed_description.Lang, Length: 14952, dtype: object

In [ ]:
from google.colab import files
files.download('FinalGameData_postProcessing.csv')

In [32]:
GameData[GameData['detailed_description.Lang']!='en']
GameData[GameData['detailed_description.Lang'].astype(str).str.contains('en')]['detailed_description.1'].values
#Nans were interpretted as [ht, zh-CN]	

array(['universe implodes interstellar nomad race get back home galaxy star violently push toward home threaten destroy nomad get catch path survive round relentless onslaught secure survival nomad game play several round round number nomad try make back home must defend swing intergalactic sword around fire projectile reflect galaxy star back origin key feature campaign span 60 mission mission increase duration difficulty random game experience random encounter implode verse simplistic control game fully play mouse huge sword attach nomad another one home sword point mouse cursor lmb reflect hazard adjacent sword rmb shoot projectile every sword toward position mouse cursor hazard collide sword destroy reflect hazard numerous type hazard require different approach hazard go home directly attack nomad multiple vector usually either tile bullet hell mechanic make round however boss round might mix match mechanic even add new one challenge you',
       'pilot rc drone target course rc ta

# Functions for preparing the text data for the LDA/NMF model application and doing coherence testing

In [4]:
#Split text into corpuses showing array of values
#Create Pre and During Covid data sets to use later:
preCOVID = GameData[GameData['Release Year'].isin([2018,2019])]
COVID = GameData[GameData['Release Year'].isin([2021,2022])]

# All Data
Descriptions_all = GameData[GameData['detailed_description.Lang'].astype(str).str.contains('en')]['detailed_description.1'].values
Reviews_all = GameData[GameData['Top Reviews by Upvotes.Lang'].astype(str).str.contains('en')]['Top Reviews by Upvotes.1'].values
Tags_all = GameData[GameData['tags.Lang'].astype(str).str.contains('en')]['tags.1'].values

# preCOVID Data
Descriptions_preCOVID = preCOVID[preCOVID['detailed_description.Lang'].astype(str).str.contains('en')]['detailed_description.1'].values
Reviews_preCOVID = preCOVID[preCOVID['Top Reviews by Upvotes.Lang'].astype(str).str.contains('en')]['Top Reviews by Upvotes.1'].values
Tags_preCOVID = preCOVID[preCOVID['tags.Lang'].astype(str).str.contains('en')]['tags.1'].values

# duringCOVID Data
Descriptions_COVID = COVID[COVID['detailed_description.Lang'].astype(str).str.contains('en')]['detailed_description.1'].values
Reviews_COVID = COVID[COVID['Top Reviews by Upvotes.Lang'].astype(str).str.contains('en')]['Top Reviews by Upvotes.1'].values
Tags_COVID = COVID[COVID['tags.Lang'].astype(str).str.contains('en')]['tags.1'].values

In [5]:
# Function to create dictionary, bag of words, and tokenized text: https://github.com/Jcc329/Text_Summarization/blob/main/notebooks/Topic_Modeling_YunpengLi.ipynb
# https://radimrehurek.com/gensim/parsing/preprocessing.html
def build_dictionary_preprocess_corpus(text_corpus):
  
  # Preprocess documents 
  tokenized_texts = preprocess_documents(text_corpus)
  
  # Create a dictionary
  dictionary = Dictionary(tokenized_texts)
  
  # Create a list of lists of bow for documents
  bow_corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

  return dictionary, bow_corpus, tokenized_texts

In [6]:
#Function for calculating coherence score: https://github.com/Jcc329/Text_Summarization/blob/main/notebooks/Topic_Modeling_YunpengLi.ipynb
# coherence_type can be 'u_mass' or 'c_v'
# algorithm can be 'LSI' or 'LDA'
def compute_coherence_score(tokenized_texts, bow_corpus, dictionary, num_topics, coherence_type, algorithm):
  if algorithm == 'LSI':
    model = LsiModel(corpus = bow_corpus, num_topics = num_topics, id2word = dictionary)
  else: # algorithm == 'LDA'
    model = LdaModel(corpus = bow_corpus, num_topics = num_topics, id2word = dictionary, passes = 2)

  coherence = CoherenceModel(model=model,
                              corpus=bow_corpus,
                              texts=tokenized_texts,
                              dictionary=dictionary,
                              coherence=coherence_type)
  return coherence.get_coherence()

In [7]:
def generate_coherence_scores(min_topics, max_topics, tokenized_texts, bow_corpus, dictionary, algorithm, coherence_type='u_mass'):
  
  coherence_scores=[]
  min_score=0
  max_score=0

  best_num_topics=1

  for k in range(min_topics, max_topics):
    score = compute_coherence_score(tokenized_texts, bow_corpus, dictionary, k, coherence_type, algorithm)
    coherence_scores.append(score)

    if coherence_type == 'u_mass':
      if score < min_score :
        min_score = score
        best_num_topics = k
    else: # coherence_type == 'c_v'
      if score > max_score:
        max_score = score
        best_num_topics= k


  return coherence_scores, best_num_topics

In [8]:
#function for plotting coherence score
import matplotlib.pyplot as plt
import matplotlib.style as style
#%matplotlib inline

def plot_coherence_scores(min_topics, max_topics, coherence_scores, coherence_type, algorithm, legend=''):
  style.use('fivethirtyeight')

  x = [int(i) for i in range(min_topics, max_topics)]

  plt.figure(figsize=(10,8))
  plt.plot(x, coherence_scores)
  plt.xlabel('Number of topics')
  plt.ylabel('Coherence Value')
  plt.title(legend + ' ' + algorithm + ' Coherence Scores by number of Topics (' + coherence_type + ')')

In [9]:
#Function to print topics
def print_topics(dictionary, bow_corpus, num_topics, algorithm, num_words=10, friendly_print=True):
  
  if algorithm == 'LSI':
    model = LsiModel(corpus=bow_corpus, num_topics=num_topics, id2word=dictionary)
  else: # algorithm == 'LDA'
    model = LdaModel(corpus=bow_corpus, num_topics=num_topics, id2word=dictionary, passes=2)

  if friendly_print:
    # Extract terms from results returned by show_topic()
    for topic_number in range(0, num_topics):
      terms = []
      for term, wt in model.show_topic(topic_number, num_words):
        terms.append(term)
    
      print('Topic ' + str(topic_number) + ':', terms)

  else:
    topics = model.print_topics(num_topics, num_words)
    for topic in topics:
      print(topic)

  return model

## Functions for NMF Modeling

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import numpy as np

In [11]:
def print_NMF_topics(tokenized_texts, max_df=0.95, min_df=1, number_topics=20, number_words_per_topic=10):
  
  text_list=[]

  for token_array in tokenized_texts:
    text_list.append(" ".join(token_array))  # concat tokens for each document to make compatible format for vectorizer.fit_transform()

  vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df, stop_words='english')
  X = vectorizer.fit_transform(text_list)

  idx_to_word = np.array(vectorizer.get_feature_names())

  # apply NMF
  nmf = NMF(n_components=number_topics, solver="mu")
  W = nmf.fit_transform(X)  # document_topics
  H = nmf.components_       # topic_terms

  for i, topic in enumerate(H):
    print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in idx_to_word [topic.argsort()[-number_words_per_topic:]]])))

#Run the models

#All Data

## Descriptions

In [ ]:
#We are using u_mass over c_v to calculate coherence because u_mass measures relationships with preceding and subsequent words and tends to perform better when compared with other coherence metrics (https://github.com/dice-group/Palmetto/issues/12)
dictionary, bow_corpus, tokenized_texts = build_dictionary_preprocess_corpus(Descriptions_all)
min_topics, max_topics = 5, 100
coherence_scores_u_mass_lda, best_num_topics_u_mass_lda = generate_coherence_scores(min_topics, max_topics, tokenized_texts, bow_corpus, dictionary, algorithm='LDA', coherence_type='u_mass')
plot_coherence_scores(min_topics, max_topics, coherence_scores_u_mass_lda, coherence_type='u_mass', algorithm='LDA')

In [ ]:
print('The best number of Description topics found by u-mass coherence score:', best_num_topics_u_mass_lda)

In [ ]:
lda_model = print_topics(dictionary, bow_corpus, best_num_topics_u_mass_lda, algorithm='LDA', num_words=15, friendly_print=True)

In [ ]:
print_NMF_topics(tokenized_texts)

## Reviews

In [ ]:
dictionary, bow_corpus, tokenized_texts = build_dictionary_preprocess_corpus(Reviews_all)
min_topics, max_topics = 5, 100
coherence_scores_u_mass_lda, best_num_topics_u_mass_lda = generate_coherence_scores(min_topics, max_topics, tokenized_texts, bow_corpus, dictionary, algorithm='LDA', coherence_type='u_mass')
plot_coherence_scores(min_topics, max_topics, coherence_scores_u_mass_lda, coherence_type='u_mass', algorithm='LDA')

In [ ]:
print('The best number of Review topics found by u-mass coherence score:', best_num_topics_u_mass_lda)

In [ ]:
lda_model = print_topics(dictionary, bow_corpus, best_num_topics_u_mass_lda, algorithm='LDA', num_words=15, friendly_print=True)

In [ ]:
print_NMF_topics(tokenized_texts)

## Tags

In [ ]:
dictionary, bow_corpus, tokenized_texts = build_dictionary_preprocess_corpus(Tags_all)
min_topics, max_topics = 5, 100
coherence_scores_u_mass_lda, best_num_topics_u_mass_lda = generate_coherence_scores(min_topics, max_topics, tokenized_texts, bow_corpus, dictionary, algorithm='LDA', coherence_type='u_mass')
plot_coherence_scores(min_topics, max_topics, coherence_scores_u_mass_lda, coherence_type='u_mass', algorithm='LDA')

In [ ]:
print('The best number of Review topics found by u-mass coherence score:', best_num_topics_u_mass_lda)

In [ ]:
lda_model = print_topics(dictionary, bow_corpus, best_num_topics_u_mass_lda, algorithm='LDA', num_words=15, friendly_print=True)

In [ ]:
print_NMF_topics(tokenized_texts)

#Pre-Covid
## Descriptions

In [ ]:
dictionary, bow_corpus, tokenized_texts = build_dictionary_preprocess_corpus(Descriptions_preCOVID)
min_topics, max_topics = 5, 100
coherence_scores_u_mass_lda, best_num_topics_u_mass_lda = generate_coherence_scores(min_topics, max_topics, tokenized_texts, bow_corpus, dictionary, algorithm='LDA', coherence_type='u_mass')
plot_coherence_scores(min_topics, max_topics, coherence_scores_u_mass_lda, coherence_type='u_mass', algorithm='LDA')

In [ ]:
print('The best number of Review topics found by u-mass coherence score:', best_num_topics_u_mass_lda)

In [ ]:
lda_model = print_topics(dictionary, bow_corpus, best_num_topics_u_mass_lda, algorithm='LDA', num_words=15, friendly_print=True)

In [ ]:
print_NMF_topics(tokenized_texts)

## Reviews

In [ ]:
dictionary, bow_corpus, tokenized_texts = build_dictionary_preprocess_corpus(Reviews_preCOVID)
min_topics, max_topics = 5, 100
coherence_scores_u_mass_lda, best_num_topics_u_mass_lda = generate_coherence_scores(min_topics, max_topics, tokenized_texts, bow_corpus, dictionary, algorithm='LDA', coherence_type='u_mass')
plot_coherence_scores(min_topics, max_topics, coherence_scores_u_mass_lda, coherence_type='u_mass', algorithm='LDA')

In [ ]:
print('The best number of Review topics found by u-mass coherence score:', best_num_topics_u_mass_lda)

In [ ]:
lda_model = print_topics(dictionary, bow_corpus, best_num_topics_u_mass_lda, algorithm='LDA', num_words=15, friendly_print=True)

In [ ]:
print_NMF_topics(tokenized_texts)

## Tags

In [ ]:
dictionary, bow_corpus, tokenized_texts = build_dictionary_preprocess_corpus(Tags_preCOVID)
min_topics, max_topics = 5, 100
coherence_scores_u_mass_lda, best_num_topics_u_mass_lda = generate_coherence_scores(min_topics, max_topics, tokenized_texts, bow_corpus, dictionary, algorithm='LDA', coherence_type='u_mass')
plot_coherence_scores(min_topics, max_topics, coherence_scores_u_mass_lda, coherence_type='u_mass', algorithm='LDA')

In [ ]:
print('The best number of Review topics found by u-mass coherence score:', best_num_topics_u_mass_lda)

In [ ]:
lda_model = print_topics(dictionary, bow_corpus, best_num_topics_u_mass_lda, algorithm='LDA', num_words=15, friendly_print=True)

In [ ]:
print_NMF_topics(tokenized_texts)

# Covid Data
## Description

In [ ]:
dictionary, bow_corpus, tokenized_texts = build_dictionary_preprocess_corpus(Descriptions_COVID)
min_topics, max_topics = 5, 100
coherence_scores_u_mass_lda, best_num_topics_u_mass_lda = generate_coherence_scores(min_topics, max_topics, tokenized_texts, bow_corpus, dictionary, algorithm='LDA', coherence_type='u_mass')
plot_coherence_scores(min_topics, max_topics, coherence_scores_u_mass_lda, coherence_type='u_mass', algorithm='LDA')

In [ ]:
print('The best number of Review topics found by u-mass coherence score:', best_num_topics_u_mass_lda)

In [ ]:
lda_model = print_topics(dictionary, bow_corpus, best_num_topics_u_mass_lda, algorithm='LDA', num_words=15, friendly_print=True)

In [ ]:
print_NMF_topics(tokenized_texts)

## Reviews

In [ ]:
dictionary, bow_corpus, tokenized_texts = build_dictionary_preprocess_corpus(Reviews_COVID)
min_topics, max_topics = 5, 100
coherence_scores_u_mass_lda, best_num_topics_u_mass_lda = generate_coherence_scores(min_topics, max_topics, tokenized_texts, bow_corpus, dictionary, algorithm='LDA', coherence_type='u_mass')
plot_coherence_scores(min_topics, max_topics, coherence_scores_u_mass_lda, coherence_type='u_mass', algorithm='LDA')

In [ ]:
print('The best number of Review topics found by u-mass coherence score:', best_num_topics_u_mass_lda)

In [ ]:
lda_model = print_topics(dictionary, bow_corpus, best_num_topics_u_mass_lda, algorithm='LDA', num_words=15, friendly_print=True)

In [ ]:
print_NMF_topics(tokenized_texts)

## Tags

In [ ]:
dictionary, bow_corpus, tokenized_texts = build_dictionary_preprocess_corpus(Tags_COVID)
min_topics, max_topics = 5, 100
coherence_scores_u_mass_lda, best_num_topics_u_mass_lda = generate_coherence_scores(min_topics, max_topics, tokenized_texts, bow_corpus, dictionary, algorithm='LDA', coherence_type='u_mass')
plot_coherence_scores(min_topics, max_topics, coherence_scores_u_mass_lda, coherence_type='u_mass', algorithm='LDA')

In [ ]:
print('The best number of Review topics found by u-mass coherence score:', best_num_topics_u_mass_lda)

In [ ]:
lda_model = print_topics(dictionary, bow_corpus, best_num_topics_u_mass_lda, algorithm='LDA', num_words=15, friendly_print=True)

In [ ]:
print_NMF_topics(tokenized_texts)

In [ ]:
## LDAvis 

In [ ]:
!pip install pyLDAvis==2.1.2

In [ ]:
import pyLDAvis.gensim

# Visualize the topics
pyLDAvis.enable_notebook()

In [ ]:
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
vis

# See Notebook 5.2 for text similarity analysis